# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [ ]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [19]:
import requests
from bs4 import BeautifulSoup


def get_book_data(url):
    """
    Получает полную информацию о книге с веб-страницы.
    
    Функция парсит страницу книги и извлекает все основные данные:
    название, цену, рейтинг, наличие, описание и технические характеристики.
    
    Args:
        url (str): URL-адрес страницы с книгой
        
    Returns:
        dict: Словарь с данными о книге, содержащий следующие ключи:
            - 'title' (str): Название книги
            - 'price' (str): Цена книги
            - 'rating' (str): Рейтинг книги (One-Five)
            - 'availability' (str): Информация о наличии
            - 'description' (str): Описание книги
            - 'product_info' (dict): Словарь с техническими характеристиками из таблицы
    """
    response = requests.get(url)

    if response.status_code != 200:
        return {"error": f"Failed to fetch page. Status code: {response.status_code}"}

    soup = BeautifulSoup(response.text, "html.parser")
    book_data = {}

    title_element = soup.find("h1")
    book_data['title'] = title_element.text.strip() if title_element else "Not found"

    price_element = soup.find("p", class_="price_color")
    book_data['price'] = price_element.text.strip() if price_element else "Not found"

    rating_element = soup.find("p", class_="star-rating")
    if rating_element:
        book_data['rating'] = rating_element["class"][1]  
    else:
        book_data['rating'] = "Not found"

    availability_element = soup.find("p", class_="instock")
    book_data['availability'] = availability_element.text.strip() if availability_element else "Not found"

    description_header = soup.find("div", id="product_description")
    if description_header:
        description_element = description_header.find_next_sibling("p")
        book_data['description'] = description_element.text.strip() if description_element else "Not found"
    else:
        book_data['description'] = "Not found"

    product_table = soup.find("table", class_="table-striped")
    book_data['product_info'] = {}

    if product_table:
        rows = product_table.find_all("tr")
        for row in rows:
            header = row.find("th")
            value = row.find("td")
            if header and value:
                book_data['product_info'][header.text.strip()] = value.text.strip()

    return book_data

In [8]:
# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': 'Â£51.77',
 'rating': 'Three',
 'availability': 'In stock (22 available)',
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [20]:
import os
import requests
from bs4 import BeautifulSoup
import time


def scrape_books(save_to_file=False, max_pages=None, delay=0.5):
    """
    Парсит все страницы каталога books.toscrape.com
    и возвращает список данных о книгах.

    Args:
        save_to_file (bool): Сохранять ли данные в файл
        max_pages (int): Максимальное количество страниц для парсинга
        delay (float): Задержка между запросами в секундах
    """
    base_url = "http://books.toscrape.com/catalogue/page-{}.html"
    all_books = []
    page_number = 1

    print(" Начало парсинга всех страниц...")

    while True:
        if max_pages and page_number > max_pages:
            print(f" Достигнут лимит в {max_pages} страниц")
            break

        url = base_url.format(page_number)
        response = requests.get(url)

        if response.status_code != 200:
            print(f"✅ Парсинг завершён. Страниц обработано: {page_number - 1}")
            break

        print(f" Обработка страницы {page_number}...")

        soup = BeautifulSoup(response.text, "html.parser")
        books = soup.select("h3 a")

        if not books:
            print(" Больше книг не найдено.")
            break

        books_count = 0
        for book in books:
            relative_link = book.get("href")
            if relative_link.startswith("../"):
                relative_link = relative_link.replace("../", "")
            book_url = f"http://books.toscrape.com/catalogue/{relative_link}"

            book_data = get_book_data(book_url)
            all_books.append(book_data)
            books_count += 1
            time.sleep(delay)

        print(f" На странице {page_number} обработано {books_count} книг")
        page_number += 1

    if save_to_file:
        artifacts_dir = "artifacts"
        os.makedirs(artifacts_dir, exist_ok=True)

        file_path = os.path.join(artifacts_dir, "books_data.txt")

        with open(file_path, "w", encoding="utf-8") as f:
            for book in all_books:
                f.write(str(book) + "\n\n")

        print(f" Данные сохранены в файл: {file_path}")
        print(f" Всего собрано данных о {len(all_books)} книгах с {page_number - 1} страниц")

    return all_books

In [21]:
# Проверка работоспособности функции
res = scrape_books(save_to_file=True)
print(type(res), len(res)) # и проверки

 Начало парсинга всех страниц...
📄 Обработка страницы 1...
 На странице 1 обработано 20 книг
📄 Обработка страницы 2...
 На странице 2 обработано 20 книг
📄 Обработка страницы 3...
 На странице 3 обработано 20 книг
📄 Обработка страницы 4...
 На странице 4 обработано 20 книг
📄 Обработка страницы 5...
 На странице 5 обработано 20 книг
📄 Обработка страницы 6...
 На странице 6 обработано 20 книг
📄 Обработка страницы 7...
 На странице 7 обработано 20 книг
📄 Обработка страницы 8...
 На странице 8 обработано 20 книг
📄 Обработка страницы 9...
 На странице 9 обработано 20 книг
📄 Обработка страницы 10...
 На странице 10 обработано 20 книг
📄 Обработка страницы 11...
 На странице 11 обработано 20 книг
📄 Обработка страницы 12...
 На странице 12 обработано 20 книг
📄 Обработка страницы 13...
 На странице 13 обработано 20 книг
📄 Обработка страницы 14...
 На странице 14 обработано 20 книг
📄 Обработка страницы 15...
 На странице 15 обработано 20 книг
📄 Обработка страницы 16...
 На странице 16 обработано 2

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [14]:
def job():
    """
    Функция, запускает парсинг и сохранение данных каждый день в 19:00.
    """
    print("Запуск парсинга...")

    scrape_books(save_to_file=True, max_pages=3, delay=0.3)

    print("✅ Парсинг завершён и данные сохранены.")

schedule.every().day.at("20:05").do(job)

print("Планировщик запущен. Ожидаем запуск задачи...")
print("Для остановки нажмите Ctrl+C")

try:
    while True:
        schedule.run_pending()
        time.sleep(2)
except KeyboardInterrupt:
    print("\n🛑 Планировщик остановлен пользователем")

Планировщик запущен. Ожидаем запуск задачи...
Для остановки нажмите Ctrl+C
Запуск парсинга...
Обработка страницы 1...
✅ Данные сохранены в файл books_data.txt
✅ Парсинг завершён и данные сохранены.

🛑 Планировщик остановлен пользователем


## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [4]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте
! pytest tests/test_scraper.py

============================= test session starts ==============================
platform darwin -- Python 3.13.5, pytest-8.3.4, pluggy-1.5.0
rootdir: /Users/olga_goncharova/MIPT_dz_python_lms/Scraper/notebooks
plugins: anyio-4.7.0
collected 0 items                                                              

============================ no tests ran in 0.00s =============================
ERROR: file or directory not found: tests/test_scraper.py



In [5]:
"Не находит директорию с тестами, поэтому добавила файл в директорию image_test_PASS"



'Не находит директорию с тестами, поэтому добавила файл в директорию image_test_PASS'

## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```